In [23]:
import pandas as pd
data = pd.read_csv('Merged_Salary_data.csv')
print(data.shape)
#data.head()

(4488, 13)


In [24]:
# Check if there are any completely duplicate rows
duplicates = data.duplicated()

# Print the number of duplicate rows
print(f"Number of duplicate rows: {duplicates.sum()}")

Number of duplicate rows: 344


In [25]:
# Find all duplicate rows (including the first occurrences)
duplicates = data[data.duplicated(keep=False)]

# Sort the duplicate rows by all columns
duplicates_sorted = duplicates.sort_values(by=data.columns.tolist())

# Print the result
#duplicates_sorted

In [26]:
data = data.drop_duplicates()

In [27]:
data.shape

(4144, 13)

In [28]:
# Drop 'salary', 'Cleaned Salary', 'Expanded Salary' 
data = data.drop(columns=['salary', 'Cleaned Salary', 'Expanded Salary'])

In [29]:
data.head()

,Title,Company Name,Location,Via,Description,Job Highlights,Date Posted,Schedule Type,Min_Salary,Max_Salary
0,Data Scientist,Apple,"Austin, TX",Careers At Apple,"Summary\nPosted: May 3, 2024\n\nRole Number:20...","expert in advanced analytics, you are passion...",NaN,Full-time,NaN,NaN
1,Data Scientist,Select Minds LLC,"Dallas, TX",LinkedIn,Benefits/Perks\n• Competitive Compensation\n• ...,e ideal candidate has an analytical mind and a...,13 hours ago,Full-time,NaN,NaN
2,Principal Data Scientist,MD Anderson,"Houston, TX",MD Anderson - MD Anderson Cancer Center,The mission of The University of Texas M. D. A...,in-depth understanding of machine learning al...,NaN,Full-time,NaN,NaN
3,Data Scientist - Causal Discovery,CPChem,"The Woodlands, TX",Careers At Chevron Phillips - Chevron Phillips...,"You may not realize it, but you’ve likely used...",is position is a great opportunity for a profe...,9 days ago,Full-time,NaN,NaN
4,Senior Data Scientist-Remote,Baker Hughes,"Houston, TX (+1 other)",Baker Hughes,Senior Data Scientist-Remote\n\nAre you passio...,ve a Masters Degree in Computer Science or “ST...,5 days ago,Full-time,104000.0,150000.0


In [30]:
# Check for missing values in each column of the dataset
missing_values = data.isnull().sum()

# Display the columns with their respective counts of missing values
missing_values

Title                0
Company Name         0
Location             4
Via                  0
Description          0
Job Highlights     279
Date Posted       1604
Schedule Type      861
Min_Salary        3279
Max_Salary        3279
dtype: int64

In [31]:
import re
import pandas as pd

# define vocabulary
synonym_dict = {
    "Python": ["Python", "python3", "py"],
    "Java": ["Java", "JVM"],
    "SQL": ["SQL", "MySQL", "PostgreSQL", "SQLite", "sql databases"],
    "Visualization Tools": ["data visualization", "dashboards", "plots", "charts", "graphs"],
    "Cloud Services": ["AWS", "Amazon Web Services", "GCP", "Azure", "Google Cloud", "Microsoft Azure"]
}

# Function to match synonyms and extract main keywords
def synonym_matching(text, synonyms):
    found_terms = set() 
    for key, words in synonyms.items():
        for word in words:
            if re.search(rf"(?i)\b{re.escape(word)}\b", text):
                found_terms.add(key)
                break  
    return ", ".join(found_terms) if found_terms else None

# Function to extract combined information, ignoring case and duplicates
def extract_combined_info(row, pattern, synonyms=None):
    combined_text = f"{str(row['Description'])} {str(row['Job Highlights'])}"
    combined_text = re.sub(r"\s+", " ", combined_text.strip()) 

    
    matches = set(re.findall(pattern, combined_text))

    
    if synonyms:
        synonym_matches = synonym_matching(combined_text, synonyms)
        if synonym_matches:
            matches.update(synonym_matches.split(", "))

    return ", ".join(matches) if matches else None

#Define the main regex patterns.
programming_languages = r"(?i)\b(Python|R|Java|JavaScript|SQL|C\+\+|C#|Julia|Go|Golang|Scala|MATLAB)\b"
visualization_tools = r"(?i)\b(Tableau|Power BI|Matplotlib|Seaborn|Plotly|D3\.js|ggplot2|QlikView|Qlik Sense|Bokeh|Altair)\b"
databases_tools = r"(?i)\b(MySQL|PostgreSQL|MongoDB|Redis|SQLite|Elasticsearch|Hadoop|Spark|Cassandra|Hive)\b"
ml_libs = r"(?i)\b(TensorFlow|PyTorch|scikit-learn|Keras|XGBoost|LightGBM|Pandas|NumPy|SciPy|NLTK|SpaCy)\b"
cloud_services = r"(?i)\b(AWS|Microsoft Azure|Google Cloud Platform|IBM Cloud|Oracle Cloud|Alibaba Cloud|Firebase|Heroku|Kubernetes|Docker)\b"
education = r"(?i)\b(Bachelor'?s?|BA|BSc|Master'?s?|MA|MSc|MBA|PhD|Doctor(?:ate)?\s*(?:of)?|Associate'?s?|Diploma|High\s*School|GED|Postgraduate|Undergraduate)\b"
#experience = r"(?i)(?:\b(?:at\s*least|minimum|more\s*than|over|around|approximately|about|up\s*to)?\s*(?:[1-3]?[0-9])\s*(?:\+|-)?\s*(?:years?|yrs?)\s*(?:of\s*)?(?:experience|exp)\b)"
experience = r"(?i)(?:\b(?:at\s*least|minimum|more\s*than|over|around|approximately|about|up\s*to)?\s*(?:[1-3]?[0-9]|(?:one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty|thirty))\s*(?:\+|-)?\s*(?:years?|yrs?)\s*(?:of\s*)?(?:experience|exp)\b)"
soft_skills = r"(?i)\b(communication(?:\s*skills)?|presentation(?:\s*skills)?|teamwork|collaboration|problem[-\s]*solving|critical\s*thinking|adaptability|emotional\s*intelligence|time\s*management|leadership|creativity|conflict\s*resolution|decision[-\s]*making|interpersonal\s*skills|negotiation|self[-\s]*motivation)\b"
responsibilities = r"(?i)(building dashboards|creating reports|analyzing trends|data cleaning|data wrangling|predictive modeling)"
industries = r"(?i)\b(healthcare|finance|technology|manufacturing)\b"
#industries = r"(?i)\b(health\s*care|healthcare|finance|financial\s*services|technology|tech|manufacturing|production|retail|education|energy|utilities|telecom|transportation|logistics|automotive|construction|real\s*estate|pharmaceutical|biotechnology|media|entertainment|hospitality|tourism|agriculture|food\s*&\s*beverage|government|public\s*sector|aerospace|defense)\b"
domains = r"(?i)\b(marketing analytics|supply chain optimization)\b"
work_modes = r"(?i)\b(remote(?:ly)?|hybrid(?:\s*model)?|on[-\s]*site|work\s*from\s*home|wfh|in[-\s]*office|telecommute|flexible\s*work|virtual(?:\s*work)?)\b"
benefits = r"(?i)\b(health insurance|401\(k\)|paid time off)\b"


# Apply to all data columns with support for synonym matching
data['Programming_Languages'] = data.apply(
    lambda row: extract_combined_info(row, programming_languages, synonym_dict), axis=1
)
data['Visualization_Tools'] = data.apply(
    lambda row: extract_combined_info(row, visualization_tools), axis=1
)
data['Databases_and_Big_Data_Tools'] = data.apply(
    lambda row: extract_combined_info(row, databases_tools), axis=1
)
data['ML_and_Data_Science_Libraries'] = data.apply(
    lambda row: extract_combined_info(row, ml_libs), axis=1
)
data['Cloud_Services'] = data.apply(
    lambda row: extract_combined_info(row, cloud_services, synonym_dict), axis=1
)
data['Education'] = data.apply(
    lambda row: extract_combined_info(row, education), axis=1
)
data['Experience'] = data.apply(
    lambda row: extract_combined_info(row, experience), axis=1
)
data['Soft_Skills'] = data.apply(
    lambda row: extract_combined_info(row, soft_skills), axis=1
)
data['Responsibilities'] = data.apply(
    lambda row: extract_combined_info(row, responsibilities), axis=1
)
data['Industries'] = data.apply(
    lambda row: extract_combined_info(row, industries), axis=1
)
data['Domains'] = data.apply(
    lambda row: extract_combined_info(row, domains), axis=1
)
data['Work_Mode'] = data.apply(
    lambda row: extract_combined_info(row, work_modes), axis=1
)
data['Benefits'] = data.apply(
    lambda row: extract_combined_info(row, benefits), axis=1
)




In [32]:
data.head()

,Title,Company Name,Location,Via,Description,Job Highlights,Date Posted,Schedule Type,Min_Salary,Max_Salary,...,ML_and_Data_Science_Libraries,Cloud_Services,Education,Experience,Soft_Skills,Responsibilities,Industries,Domains,Work_Mode,Benefits
0,Data Scientist,Apple,"Austin, TX",Careers At Apple,"Summary\nPosted: May 3, 2024\n\nRole Number:20...","expert in advanced analytics, you are passion...",NaN,Full-time,NaN,NaN,...,None,"Python, Kubernetes, Docker, SQL","Bachelor, MSc",5+ Years of experience,"communication skills, collaboration, leadership",data wrangling,None,None,None,None
1,Data Scientist,Select Minds LLC,"Dallas, TX",LinkedIn,Benefits/Perks\n• Competitive Compensation\n• ...,e ideal candidate has an analytical mind and a...,13 hours ago,Full-time,NaN,NaN,...,None,"Visualization Tools, Python, SQL",bachelor,None,communication skills,None,None,None,None,None
2,Principal Data Scientist,MD Anderson,"Houston, TX",MD Anderson - MD Anderson Cancer Center,The mission of The University of Texas M. D. A...,in-depth understanding of machine learning al...,NaN,Full-time,NaN,NaN,...,"PyTorch, TensorFlow, Scikit-learn","AWS, Python, Cloud Services","Master's, PhD, Bachelor's, Doctorate","five years experience, Seven years of experien...","decision-making, collaboration, Collaboration,...",None,healthcare,None,Remote,None
3,Data Scientist - Causal Discovery,CPChem,"The Woodlands, TX",Careers At Chevron Phillips - Chevron Phillips...,"You may not realize it, but you’ve likely used...",is position is a great opportunity for a profe...,9 days ago,Full-time,NaN,NaN,...,None,"Python, Cloud Services, Microsoft Azure, AWS, SQL","Masters, PhD",None,"communication skills, decision-making",None,"manufacturing, finance",None,None,None
4,Senior Data Scientist-Remote,Baker Hughes,"Houston, TX (+1 other)",Baker Hughes,Senior Data Scientist-Remote\n\nAre you passio...,ve a Masters Degree in Computer Science or “ST...,5 days ago,Full-time,104000.0,150000.0,...,"TensorFlow, Keras",Python,Masters,None,communication skills,None,"technology, Technology",None,"remotely, remote, Remote",None


In [33]:
# Adjusting the output path to ensure it's valid within the current environment
output_path = "extracted_final_data.csv"

# Save the extracted data to a new CSV file
data.to_csv(output_path, index=False)

# Confirm the output path
output_path


'extracted_final_data.csv'